In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bluff,NZ,2022-04-13 10:38:00,-46.6000,168.3333,7.11,75,56,5.42
1,1,Zhaotong,CN,2022-04-13 10:38:00,27.3167,103.7167,20.06,58,38,2.52
2,2,Severo-Kurilsk,RU,2022-04-13 10:38:01,50.6789,156.1250,-1.44,88,100,4.87
3,3,Saint-Philippe,RE,2022-04-13 10:38:01,-21.3585,55.7679,25.77,86,84,6.39
4,4,Port Alfred,ZA,2022-04-13 10:38:01,-33.5906,26.8910,25.10,76,5,2.24


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
#print(locations)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
#print(max_temp)
#print(city_data_df.loc[(city_data_df['Max Temp']<=0)])
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, temps)
# OR we can replace the whole code in the previous cell in the same line inside the gmaps heatmap_layer:
#heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 22
What is the maximum temperature you would like for your trip? 28


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_city_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Saint-Philippe,RE,2022-04-13 10:38:01,-21.3585,55.7679,25.77,86,84,6.39
4,4,Port Alfred,ZA,2022-04-13 10:38:01,-33.5906,26.8910,25.10,76,5,2.24
6,6,Grand-Santi,GF,2022-04-13 10:38:02,4.2500,-54.3833,23.03,97,83,1.13
7,7,Bredasdorp,ZA,2022-04-13 10:38:02,-34.5322,20.0403,24.55,53,34,5.53
8,8,Aksu,CN,2022-04-13 10:38:02,41.1231,80.2644,24.73,15,53,1.88


In [9]:
preferred_city_df.count()

City_ID       158
City          158
Country       156
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [13]:
preferred_city_df_null = preferred_city_df.loc[preferred_city_df["Country"].isnull()==True]
preferred_city_df_null

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
261,261,Walvis Bay,NaN,2022-04-13 10:44:13,-22.9575,14.5053,23.53,49,99,2.06
313,313,Luderitz,NaN,2022-04-13 10:46:27,-26.6481,15.1594,22.16,59,20,6.02


In [14]:
preferred_city_df = preferred_city_df.dropna()
preferred_city_df.count()

City_ID       156
City          156
Country       156
Date          156
Lat           156
Lng           156
Max Temp      156
Humidity      156
Cloudiness    156
Wind Speed    156
dtype: int64

In [18]:
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Saint-Philippe,RE,25.77,-21.3585,55.7679,
4,Port Alfred,ZA,25.10,-33.5906,26.8910,
6,Grand-Santi,GF,23.03,4.2500,-54.3833,
7,Bredasdorp,ZA,24.55,-34.5322,20.0403,
8,Aksu,CN,24.73,41.1231,80.2644,
...,...,...,...,...,...,...
560,Moerai,PF,26.84,-22.4333,-151.3333,
561,Sibolga,ID,27.43,1.7427,98.7792,
569,Jaicos,BR,26.33,-7.3592,-41.1378,
574,Kupang,ID,27.16,-10.1667,123.5833,


In [35]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [36]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [39]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Saint-Philippe,RE,25.77,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
4,Port Alfred,ZA,25.10,-33.5906,26.8910,The Halyards Hotel
6,Grand-Santi,GF,23.03,4.2500,-54.3833,Hotel La Villa Couacou
7,Bredasdorp,ZA,24.55,-34.5322,20.0403,Bredasdorp Country Manor
8,Aksu,CN,24.73,41.1231,80.2644,Pudong Holiday Hotel
17,Caraballeda,VE,23.04,10.6112,-66.8522,Hotel 15 Letras
19,Itaituba,BR,23.55,-4.2761,-55.9836,Hotel Campos
22,Nabire,ID,26.31,-3.3667,135.4833,Hotel Nusantara I
34,Rikitea,PF,25.86,-23.1203,-134.9692,People ThankYou
35,Saravena,CO,22.00,6.9538,-71.8752,Hotel Diamante Saravena


In [55]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, max_temp, dissipating=False, max_intensity=75, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
# Add a heatmap of temperature for the vacation spots and marker for each city
locations = hotel_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap and marker variables.
heat_layer = gmaps.heatmap_layer(locations, max_temp, dissipating=False, max_intensity=75, point_radius=4)
markers = gmaps.marker_layer(locations)
# Add the heatmap and marker layers.
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
info_box_content=hotel_info

In [60]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))